# Introduction to Prompt Engineering

해당 노트북을 통해서 프롬프트 엔지니어링 기법에 대해서 알아보겠습니다.

## 실습 환경 설정
실습 진행에 필요한 라이브러리를 설치하고 Bedrock API 호출에 사용할 AWS SDK를 설정합니다.

In [ ]:
!pip install xmltodict

import pandas as pd
import numpy as np
import boto3
import json

bedrock = boto3.client(
    'bedrock',
    region_name="us-east-1"
)

bedrock_runtime = boto3.client(
    'bedrock-runtime',
    region_name="us-east-1"
)

def cprint(text):
    print(f'\033[1m\033[94m{text}')

Amazon Bedrock에서 제공하는 모델 목록을 확인

In [ ]:
response = bedrock.list_foundation_models(
)

fm = pd.DataFrame.from_records(response['modelSummaries'])
fm = fm.drop(columns=['modelArn', 'modelId', 'responseStreamingSupported', 'customizationsSupported', 'inferenceTypesSupported', 'modelLifecycle'])
fm = fm.drop_duplicates(subset=['modelName'])
fm = fm.rename(columns={"modelName": "Model", "providerName": "Provider", "inputModalities": "Input", "outputModalities": "Output"})
fm = fm[['Provider', 'Model', 'Input', 'Output']]
fm.style.hide(axis="index")

## 프롬프트 기초
실습에서 사용할 Claude 모델 호출에 필요한 변수 및 프롬프트 구조에 대해서 알아보겠습니다.

Claude 모델을 호출하는 함수 생성 - https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html

In [ ]:
def invoke_claude(prompt):
    body = json.dumps({"prompt": prompt})
    respnose = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2:1',
        body=body
    )
    return respnose

모델 호출 테스트

In [ ]:
cprint(invoke_claude('대한민국의 수도는?'))

Claude 모델 추론에 필요한 변수 확인 - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html

Claude 모델을 호출하는 함수 수정

In [ ]:
def invoke_claude(prompt):
    body = json.dumps(
        {
            "prompt": prompt,
            "max_tokens_to_sample": 500
        }
    )
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2:1',
        body=body
    )
    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')

모델 호출 테스트

In [ ]:
cprint(invoke_claude('대한민국의 수도는?'))

변수로 입력하는 문자열을 Claude가 요구하는 프롬프트 양식으로 변환 후 모델을 호출하는 함수 생성

In [ ]:
def invoke_claude_auto_format(prompt):
    body = json.dumps(
        {
            "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
            "max_tokens_to_sample": 500
        }
    )
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2:1',
        body=body
    )
    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')

모델 호출 테스트

In [ ]:
cprint(invoke_claude_auto_format('대한민국의 수도는?'))

대화 기능이 가능한지 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_auto_format('인구는 얼마인가요?'))

프롬프트와 응답을 조합해서 대화 형식으로 출력해보겠습니다.

In [ ]:
prompt = '대한민국의 수도는?'
response = invoke_claude_auto_format(prompt)
dialogue = f'\n\nHuman: {prompt}\n\nAssistant:{response}'
cprint(dialogue)

이전 대화 내용을 프롬프트에 추가해서 대화 기능이 가능한지 확인해 보겠습니다.

In [ ]:
def chat_claude(prompt, previous_dialogue):
    if previous_dialogue:
        prompt = f'{previous_dialogue}\n\nHuman: {prompt}\n\nAssistant:'
        cprint(prompt)
    else:
        prompt = f'\n\nHuman: {prompt}\n\nAssistant:'
        
    body = json.dumps(
        {
            "prompt": prompt,
            "max_tokens_to_sample": 500
        }
    )
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2:1',
        body=body
    )
    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')


cprint(chat_claude('인구는 얼마인가요?', dialogue))

## Few-Shot Prompting
거대 언어 모델을 통해서 복잡하거나 불분명한 작업을 수행하고자 할때 정확도가 떨어지거나 특정 패턴으로 응답을 받고 싶다면 작업에 대한 예시나 정보를 프롬프트에 추가해서 응답의 정확도를 높힐수 있습니다.

프롬프트에 예제를 넣을수 있도록 새로운 함수를 생성하겠습니다.

In [ ]:
def invoke_claude_raw(prompt):
    body = json.dumps(
        {
            "prompt": prompt,
            "max_tokens_to_sample": 500
        }
    )
    response = bedrock_runtime.invoke_model(
        modelId='anthropic.claude-v2:1',
        body=body
    )
    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')

여행 관련 애플리케이션을 서비스하는 스타트업에서 사용자 경혐을 극대화하기 위해서 자연어 기반으로 항공권 검색이 가능한 기능을 개발한다고 가정하겠습니다.
사용자가 검색창에서 아래와 같은 문구를 입력해서 항공권 검색이 가능하도록 하고자 합니다.

"인천에서 제주도로 가고 싶습니다"

하지만 API 서버를 통해서 항공권 검색을 하려면 출발지 및 도착지 공항의 코드를 변수로 입력해야 합니다.

위의 문구를 프롬프트로 입력해서 LLM이 어떤 응답을 하지는 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw('''
Human: 인천에서 제주도로 가고 싶습니다

Assistant:
'''))

LLM이 인천에서 제주도로 가는 다양한 방법에 대해서 응답을 주지만 저희가 구현하고자 하는 기능은 아닙니다.

**Zero-Shot Prompting**은 별도의 예제를 프롬프트에 포함하지 않는 방식입니다. 아래의 프롬프트를 입력해서 LLM을 통해서 각 공항의 코드를 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw('''
Human: 인천공항과 제주공항의 코드는 무엇인가요?

Assistant:
'''))

Zero-Shot Prompting을 통해서 Claude 모델을 통해서 공항 코드를 확인할 수 있음을 확인했습니다. 만약 사용하고자 하는 LLM이 Zero-Shot Prompting을 통해서 원하고자 하는 정보을 제공해주지 못한다면 Few-Shot Prompting을 통해서 원하는 정보를 얻을수 있는 가능성이 낮기 때문에 RAG 혹은 파인튜닝을 고려하셔야 합니다.

하지만 위에서 사용한 프롬프트는 사용자가 실제로 입력할 문구일 가능성이 낮기 때문에 다른 방식으로 공항 코드를 확인해야 할것입니다.

**One-Shot Prompting**은 하나의 예제를 프롬프트에 추가해서 LLM이 프롬프트를 통해서 추가적인 정보를 받도록 하는 기법입니다. 아래의 프롬프트를 입력해서 LLM을 통해서 각 공항의 코드를 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw('''
Human: 인천에서 뉴욕으로 가고 싶습니다.

Assistant: 공항 코드 [ICN,JFK]

Human: 인천에서 도쿄로 가고 싶습니다.

Assistant:
'''))

응답에 공항 코드가 포함되어 있을수도 있고 없을수도 있습니다. 만약 응답에 공항 코드가 포함되어 있다면 LLM이 프롬프트를 통해서 *추가적인 정보*를 확인해서 응답을 주었음이 검증되었습니다. 하지만 공항 코드가 포함되어 있더라도 원하는 패턴대로 출력이 되지 않는다면 별도의 자연어 처리가 필요하므로 실효성이 떨어집니다.

**Few-Shot Prompting**은 복수의 예제를 프롬프트에 추가해서 LLM이 프롬프트를 통해서 추가적인 정보를 받도록 하는 기법입니다. 아래의 프롬프트를 입력해서 LLM을 통해서 각 공항의 코드를 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw('''
Human: 인천에서 김해로 가고 싶습니다.

Assistant: 공항 코드 [ICN,PUS]

Human: 인천에서 제주로 가고 싶습니다.

Assistant: 공항 코드 [ICN,CJU]

Human: 인천에서 하노이로 가고 싶습니다.

Assistant: 공항 코드 [ICN,HAN]

Human: 인천에서 뉴욕으로 가고 싶습니다.

Assistant: 공항 코드 [ICN,JFK]

Human: 인천에서 도쿄로 가고 싶습니다.

Assistant:
'''))

응답에 공항 코드가 포함되어 있고 원하는 패턴대로 [출발 공항코드,도착 공항코드] 출력이 된다면 Few-Shot Prompting을 통해서 원하는 기능을 구현할수 있음이 검증되었습니다.

하지만 LLM은 항상 동일한 응답을 주지 않기 때문에 해당 방법이 항상 동작한다는 보장은 없으면 동일한 함수를 다수 호출해서 정확도를 높일수 있습니다. 추가로 응답에 원하는 패턴이 추가되어 있지만 API 서버에 변수로 넣기 위해서는 프로그래밍적인 요소를 추가할 필요가 있을수도 있습니다. 아래의 함수를 실행해서 응답에서 공항 코드만 축출해 보겠습니다.

In [ ]:
import re
responses = []
for _ in range(3):
    responses.append(invoke_claude_raw('''
Human: 인천에서 김해로 가고 싶습니다.

Assistant: 공항 코드 [ICN,PUS]

Human: 인천에서 제주로 가고 싶습니다.

Assistant: 공항 코드 [ICN,CJU]

Human: 인천에서 하노이로 가고 싶습니다.

Assistant: 공항 코드 [ICN,HAN]

Human: 인천에서 뉴욕으로 가고 싶습니다.

Assistant: 공항 코드 [ICN,JFK]

Human: 인천에서 도쿄로 가고 싶습니다.

Assistant:
    '''))
    
matches = []
for res in responses:
    res = ''.join(res.split())
    match = re.findall('\[[A-Z]{3},[A-Z]{3}\]', res)
    matches.extend(match)

airport_code = list(set(matches))
cprint(airport_code)

## Instruction

프롬프트에 모델이 수행할 특정 작업이니 지시를 상세하게 명시할수록 정확도는 높아집니다. 아래의 프롬프트를 입력해서 LLM을 통해서 각 공항의 코드를 확인해 보겠습니다.

사용자는 "인천에서 도쿄로 가고 싶습니다"를 입력하지만 애플리케이션 레벨에서 "출발지와 도착지 공항의 코드를 JSON 형식으로 출력해주세요"를 추가하도록 하겠습니다.

In [ ]:
user_input = '인천에서 도쿄로 가고 싶습니다.'
instruction = '출발지와 도착지 공항의 코드를 JSON 형식으로 출력해주세요.'

cprint(invoke_claude_raw(f'''
Human: {user_input} {instruction}

Assistant:
'''))

동일한 프롬프트를 여러번 호출해서 항상 동일한 키값을 가지는 JSON이 응답에 포함되는지 확인해 보겠습니다.

In [ ]:
for i in range(3):    
    cprint(invoke_claude_raw(f'''
Human: {user_input} {instruction}

Assistant:
''').split('```json')[1].split('```')[0])

프롬프트의 출력 지시 부분에 JSON의 키 값을 지정해 보겠습니다.

In [ ]:
instruction = '출발지와 도착지 공항의 코드를 JSON 형식으로 출력해주세요. 출발지 공항 코드의 키는 origin이고 도착지 공항 코드의 키는 destination 입니다.'

for i in range(3):    
    cprint(invoke_claude_raw(f'''
Human: {user_input} {instruction}

Assistant:
''').split('```json')[1].split('```')[0])

## Function Calling / Agent

Function Calling을 통해서 프롬프트에 명시된 작업에 적합한 함수와 변수를 추천받을수 있습니다. LLM으로부터 추천받은 함수를 애플리케이션에서 실행한 다음 나온 결과를 다시 프롬프트에 추가해서 LLM을 호출해서 사용자에게 응답을 줄수가 있습니다. 이와 같은 워크플로우를 조율/관리(Orchestrate)하는 애플리케이션 로직을 Agent라고 부릅니다.

위의 예제에서는 검색창에 사용자가 원하는 비행 일정을 입력하면 LLM을 통해서 공항 코드를 축출한 후에 API 서버를 호출해서 항공권 검색이 가능하도록 하였습니다. 이 경우에는 검색창이 특정 함수와 일대일 매칭이 됩니다. 하지만 지능형 챗봇같이 하나의 기능이 아니라 다수의 기능, 예를 들면 항공권 검색, 예약, 취소, 수정 등등의 작업을 수행하고자 한다면 사용자 입력을 분석해서 어떤 함수를 호출할지는 결정해야 합니다.

아래와 같은 프롬프트 템플릿을 정의하겠습니다.

In [ ]:
function_call_template = '''
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. You can call one function only.
4. The Results of the function will be provided in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>list_flights</tool_name>
<parameters>
<name>origin</name>
<name>destination</name>
</parameters>
</tool_description>

<tool_description>
<tool_name>update_booking</tool_name>
<parameters>
<name>booking_id</name>  
</parameters>
</tool_description>
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem?

Assistant:
'''


def get_function_call_prompt(template, user_input):
    prompt = template.replace('{user_input}', user_input)
    return prompt    

사용자가 채팅창에 "인천에서 도쿄로 가는 항공권을 예악하고 싶습니다"라는 문자열을 입력했을때 어떤 함수를 호출하도록 지시하는지 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw(
    get_function_call_prompt(function_call_template, "인천에서 도쿄로 가는 항공권을 예악하고 싶습니다."))
)

위에서 사용한 템플릿은 영문으로 되어 있어서 답변도 영문으로 출력이 됩니다. 템플릿을 한글로 바꾸는 대신 응답을 한글로 하도록 지시해보겠습니다.

In [ ]:
function_call_template = '''
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. You can call one function only.
4. The Results of the function will be provided in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>list_flights</tool_name>
<parameters>
<name>origin</name>
<name>destination</name>
</parameters>
</tool_description>

<tool_description>
<tool_name>update_booking</tool_name>
<parameters>
<name>booking_id</name>  
</parameters>
</tool_description>
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem? Please provide the answer in Korean.

Assistant:
'''

cprint(invoke_claude_raw(
    get_function_call_prompt(function_call_template, "인천에서 도쿄로 가는 항공권을 예악하고 싶습니다."))
)

변수에 공항 코드가 아니라 출발지/도착지 도시 이름이 나오는 경우도 있으므로 템플릿 수정하도록 하겠습니다.

In [ ]:
function_call_template = '''
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. You can call one function only.
4. The results of the function will be provided in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>list_flights</tool_name>
<parameters>
<name>origin_airport_code</name>
<name>destination_airport_code</name>
</parameters>
</tool_description>

<tool_description>
<tool_name>update_booking</tool_name>
<parameters>
<name>booking_id</name>  
</parameters>
</tool_description>
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem? Please provide the answer in Korean.

Assistant:
'''

cprint(invoke_claude_raw(
    get_function_call_prompt(function_call_template, "인천에서 도쿄로 가는 항공권을 예악하고 싶습니다."))
)

예약한 항공권이 있다고 가정하고 사용자가 항공권 변경을 원할때 어떤 함수를 호출하도록 지시하는지 확인해 보겠습니다.

In [ ]:
cprint(invoke_claude_raw(
    get_function_call_prompt(function_call_template, "예약한 항공권의 일정을 변경하고 싶습니다."))
)

예약 정보를 불러오는 함수를 템플릿에 추가하겠습니다.

In [ ]:
function_call_template = '''
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. You can call one function only.
4. The results of the function will be provided in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>list_flights</tool_name>
<parameters>
<name>origin_airport_code</name>
<name>destination_airport_code</name>
</parameters>
</tool_description>

<tool_description>
<tool_name>update_booking</tool_name>
<parameters>
<name>booking_id</name>  
</parameters>
</tool_description>
</tools>

<tool_description>
<tool_name>get_booking</tool_name>
<parameters>
<name>booking_id</name>  
</parameters>
</tool_description>
</tools>

<tool_description>
<tool_name>list_bookings</tool_name>
<parameters>
<name>user_id</name>  
</parameters>
</tool_description>
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem? Please provide the answer in Korean.

Assistant:
'''

사용자가 애플리케이션에 로그인을 하면 사용자 정보를 확인할수 있으므로 프롬프트에 사용자 정보를 추가해보겠습니다.

In [ ]:
cprint(invoke_claude_raw(
    get_function_call_prompt(function_call_template, "예약한 항공권의 일정을 변경하고 싶습니다. user_id=1234"))
)

템플릿에 명시한 함수들을 생성 하겠습니다.

In [ ]:
def list_flights(origin_airport_code, destination_airport_code):
    response = [
        {
            "flight_code": "KE123",
            "depart_time": "11:25",
            "price": "400,000"
        },
        {
            "flight_code": "OZ999",
            "depart_time": "13:00",
            "price": "500,000"
        },
    ] 
    return response


def get_booking(booking_id):
    response = {
        "booking_id": 123123123,
        "type": "flight",
        "date": "2024-01-01"
    }
    return response


def list_bookings(user_id):
    response = {
        "active_booking_ids": [
            "123123123"
        ]
    }
    return response


def update_booking(booking_id):
    response = {
        "booking_id": booking_id,
        "updated": True,
        "changes": [
            {
                "date": "2024-01-11" 
            }
        ]
    }
    return response


def call_function(tool_name, parameters):
    func = globals()[tool_name]
    output = func(**parameters)
    return output

애플리케이션 로직을 수행할 Agent 생성

In [ ]:
import xmltodict

def agent(prompt, output):

    # LLM 모델이 사용자가 요청한 작업에 대한 최종 답변을 주었는지 확인
    done = False
    if '<answer>' in output:
        answer = output.split('<answer>')[1]
        answer = answer.split('</answer>')[0]
        done = True
        return done, answer
    
    # 최종 답변을 주지 않았다면 응답에 포함된 함수를 호출
    else:

        # 응답에서 호출할 함수 및 변수 확인
        function_xml = output.split('<function_calls>')[1]
        function_xml = function_xml.split('</function_calls>')[0]
        function_dict = xmltodict.parse(function_xml)
        func_name = function_dict['invoke']['tool_name']
        parameters = function_dict['invoke']['parameters']

        # 함수 호출
        func_response = call_function(func_name, parameters)

        # 호출한 함수가 반환한 정보를 축출
        func_response_str = '\n\nHuman: Here is the result from your function call\n\n'
        func_response_str = func_response_str + f'<function_results>\n{func_response}\n</function_results>'
        func_response_str = func_response_str + '\n\nIf you know the answer, say it. If not, what is the next step?\n\nAssistant:'

        # 호출한 함수가 반환한 정보를 프롬프트에 추가
        prompt = prompt + output + func_response_str
    return done, prompt

에이전트를 통해서 항공권 검색

In [ ]:
prompt = get_function_call_prompt(function_call_template, "인천에서 도쿄로 가는 항공권편 정보를 알고 싶습니다.")

while True:
    output = invoke_claude_raw(prompt).strip()
    done, prompt = agent(prompt, output)
    print(prompt)
    if done:
        cprint(prompt)
        break

에이전트를 통해서 예약 목록 확인

In [ ]:
prompt = get_function_call_prompt(function_call_template, "예약 목록을 확인하고 싶습니다. user_id=1234")

while True:
    output = invoke_claude_raw(prompt).strip()
    done, prompt = agent(prompt, output)
    print(prompt)
    if done:
        cprint(prompt)
        break

에이전트를 통해서 예약 수정

In [ ]:
prompt = get_function_call_prompt(function_call_template, "예약을 수정하고 싶습니다. booking_id=123123123")

while True:
    output = invoke_claude_raw(prompt).strip()
    done, prompt = agent(prompt, output)
    if done:
        cprint(prompt)
        break